# Boring Definition

In [2]:
# The Beginning
# Define the functios and load the data.
import collections
from rich import inspect
from rich import print
from rich.console import Console
import time
import csv
from IPython.display import Image, display
import datetime
import io
import math
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import openpyxl
import pandas as pd
import requests
import seaborn as sns
import warnings
from rich import print
from rich.console import Console
from rich.table import Table
from rich.progress import track
from rich.panel import Panel
from rich.text import Text
from rich import box
from rich import color
from folium import FeatureGroup, Tooltip
from folium.features import DivIcon
from folium.plugins import FastMarkerCluster, Fullscreen
from haversine import haversine, Unit
from IPython.display import HTML
# import display for terminal to show the map
from IPython.display import display
from math import ceil
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from regex import F
from requests.structures import CaseInsensitiveDict
from scipy.spatial import ConvexHull
from tqdm import tqdm
from urllib import response
from urllib.parse import quote_plus
import folium
import os
import webbrowser
import time
from selenium import webdriver
from PIL import Image, ImageSequence
import os
from IPython.display import HTML, display
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import numpy as np  # For colormap or random color generation
# Function to convert HTML maps to images
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display
# Prime the Solver With a First Run
from time import sleep
import weakref
from IPython.core.display import HTML
from rich.console import Console
from rich.table import Table
from rich.layout import Layout
from rich.columns import Columns
from rich.panel import Panel

console = Console()
table = Table()

warnings.simplefilter(action="ignore", category=FutureWarning)
# # FunctionF Definitions
EARTH_RADIUS = 6371.0


def create_final_route_frame(dataset, output_directory='route_snapshots'):
    """
    Creates the final frame of a route visualization.

    Args:
        dataset (pd.DataFrame): A DataFrame containing route data with columns:
            'Latitude', 'Longitude', 'NEW RT', 'NEW DAY', 'NEW STOP'

    Returns:
        matplotlib.figure.Figure: The Matplotlib figure object representing the final frame.
    """
    sorted_dataset = dataset.sort_values(by=['NEW DAY', 'NEW STOP'])
    # Make sure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Assign a unique color to each route
    unique_routes = sorted_dataset['NEW RT'].unique()
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_routes)))
    route_colors = {rt: clr for rt, clr in zip(unique_routes, colors)}

    # Determine global latitude and longitude ranges
    lat_min, lat_max = sorted_dataset['Latitude'].min(
    ), sorted_dataset['Latitude'].max()
    lon_min, lon_max = sorted_dataset['Longitude'].min(
    ), sorted_dataset['Longitude'].max()

    # Initialize a dictionary to store cumulative route points
    cumulative_route_points = {rt: pd.DataFrame(
        columns=['Latitude', 'Longitude']) for rt in unique_routes}

    # Extract the last day and stop
    # last_day, last_stop = sorted_dataset.iloc[-1][['NEW DAY', 'NEW STOP']]

   # Get max values for day and stop for the final frame
    max_day = sorted_dataset['NEW DAY'].max()
    max_stop = sorted_dataset['NEW STOP'].max()

    # Create the final figure
    plt.figure(figsize=[8, 6])

    for rt in unique_routes:
        # Get all data for the route
        rt_data = sorted_dataset[sorted_dataset['NEW RT'] == rt]
        if not rt_data.empty:
            cumulative_route_points[rt] = rt_data[[
                'Latitude', 'Longitude']].drop_duplicates()  # Cumulative points
            plt.plot(cumulative_route_points[rt]['Longitude'], cumulative_route_points[rt]['Latitude'],
                     '-o', color=route_colors[rt], markersize=5, linewidth=2, label=f'Route {rt}')
    now = datetime.datetime.now()
    plt.title(f'Final Routes {now}')  # Title for final frame
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.xlim(lon_min - 0.05, lon_max + 0.05)
    plt.ylim(lat_min - 0.05, lat_max + 0.05)
    # plt.legend()
    now = datetime.datetime.now()
    filename = os.path.join(output_directory, f'route_snapshot_{now}.png')
    plt.savefig(filename)
    # plt.show()

    plt.close()
    return filename  # Return the filename of the saved image


def create_video_from_filenames(filenames, output_filename='route_animation.mp4', fps=10):
    """
    Creates a video from a list of image filenames.

    Args:
        filenames (list of str): A list of filenames of the images to include in the video.
        output_filename (str): The filename for the video to be saved as.
        fps (int): Frames per second for the video.
    """
    import imageio

    # Create the MP4 animation
    with imageio.get_writer(output_filename, fps=fps) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    print(f'Video saved as {output_filename}')


def create_route_animation(dataset, output_filename='route_day_stop_animation.mp4'):
    """
    Creates an animated map visualization of routes over time.

    Args:
        dataset (pd.DataFrame): A DataFrame containing route data with columns:
            'Latitude', 'Longitude', 'NEW RT', 'NEW DAY', 'NEW STOP'
        output_filename (str, optional): The filename for the MP4 output. 
                """
    sorted_dataset = dataset.sort_values(by=['NEW DAY', 'NEW STOP'])
    # Assign a unique color to each route
    unique_routes = sorted_dataset['NEW RT'].unique()
    # Change colormap as needed
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_routes)))
    route_colors = {rt: clr for rt, clr in zip(unique_routes, colors)}

    #  latitude and longitude ranges
    lat_min, lat_max = sorted_dataset['Latitude'].min(
    ), sorted_dataset['Latitude'].max()
    lon_min, lon_max = sorted_dataset['Longitude'].min(
    ), sorted_dataset['Longitude'].max()

    # Initialize a dictionary to store cumulative route points
    cumulative_route_points = {rt: pd.DataFrame(
        columns=['Latitude', 'Longitude']) for rt in unique_routes}

    filenames = []  # To store the names of the plot files

    # Iterate through each day and stop
    for (day, stop), group in sorted_dataset.groupby(['NEW DAY', 'NEW STOP']):
        plt.figure(figsize=[8, 6])  # Set figure size as needed

        for rt in unique_routes:
            # Filter for current route up to the current stop
            rt_data = sorted_dataset[(sorted_dataset['NEW RT'] == rt) & (
                sorted_dataset['NEW DAY'] <= day) & (sorted_dataset['NEW STOP'] <= stop)]
            if not rt_data.empty:
                cumulative_route_points[rt] = pd.concat([cumulative_route_points[rt], rt_data[[
                                                        'Latitude', 'Longitude']]], ignore_index=True).drop_duplicates()
                plt.plot(cumulative_route_points[rt]['Longitude'], cumulative_route_points[rt]['Latitude'],
                         '-o', color=route_colors[rt], markersize=5, linewidth=2, label=f'Route {rt}')

        plt.title(f'Up to Day {day}, Stop {stop}')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.xlim(lon_min - 0.01, lon_max + 0.01)
        plt.ylim(lat_min - 0.01, lat_max + 0.01)
        plt.legend()

        # Save the frame
        filename = f'map_day{day}_stop{stop}.png'
        plt.savefig(filename)
        filenames.append(filename)
        plt.close()

    # Create the MP4 animation
    # Adjust FPS as needed
    with imageio.get_writer('route_day_stop_animation.mp4', fps=14) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Optionally, clean up the individual PNG files
    for filename in filenames:
        os.remove(filename)

    # To display the video in Jupyter Notebook
    video_path = 'route_day_stop_animation.mp4'
    display(HTML(f"""
        <video controls>
            <source src="{video_path}" type="video/mp4">
        </video>
        """)
            )


def html_to_image(html_files, output_dir='maps_images', width=800, height=600):
    # Setup Selenium webdriver
    options = webdriver.ChromeOptions()
    options.add_argument(f'--window-size={width},{height}')
    driver = webdriver.Chrome(options=options)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = []

    for html_file in html_files:
        # Open the HTML file
        driver.get(f"file://{os.path.abspath(html_file)}")
        # time.sleep(1)  # Wait for the map to load

        # Define the output image path
        output_image = f"{output_dir}/{os.path.basename(html_file).replace('.html', '.png')}"
        image_files.append(output_image)

        # Take screenshot
        driver.save_screenshot(output_image)

    driver.quit()
    return image_files


def create_animation(image_files, output_file='map_animation.gif', duration=1000):
    images = [Image.open(image_file).convert(
        "P", palette=Image.ADAPTIVE) for image_file in image_files]
    images[0].save(output_file, save_all=True,
                   append_images=images[1:], duration=duration, loop=0)


def routes_by_days_to_html(dataset):
    # Starting the HTML content
    html_content = """
    <html>
    <head>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            table-layout: fixed; /* Add fixed table layout */
        }
        th, td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
            overflow: hidden; /* Ensures the content fits the cell */
            word-wrap: break-word; /* Ensures words do not exceed the cell width */
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
    </head>
    <body>
    """
    # Get unique routes from the dataset
    unique_routes = sorted(dataset["NEW RT"].unique())

    # Iterate over each unique route
    for route in unique_routes:
        html_content += f"<h2>Route {route} Summary:</h2>"

        # Subset the data for the current route
        subset_data = dataset[dataset["NEW RT"] == route].sort_values(
            ["NEW DAY", "NEW STOP"]
        )

        # Find the maximum number of days for this route
        max_day = subset_data["NEW DAY"].max()

        # Iterate over each group of days
        num_groups = ceil(max_day / 5)
        for group in range(num_groups):
            day_start = group * 5 + 1
            day_end = min((group + 1) * 5, max_day)
            day_headers = [
                f"Day {day_num}" for day_num in range(day_start, day_start + 5)
            ]  # Ensure 5 headers per group

            # Start a new table for each group of days
            html_content += (
                "<table><tr>"
                + "".join([f"<th>{header}</th>" for header in day_headers])
                + "</tr>"
            )

            # Determine the maximum number of stops in any day within this group for formatting
            max_stops = 0
            for day_num in range(day_start, day_end + 1):
                day_length = subset_data[subset_data["NEW DAY"]
                                         == day_num].shape[0]
                if day_length > max_stops:
                    max_stops = day_length

            # Print customer names for each day in the group
            for stop_num in range(1, max_stops + 1):
                html_content += "<tr>"
                # Include empty cells if fewer than 5 days in the group
                for day_num in range(day_start, day_start + 5):
                    day_data = subset_data[subset_data["NEW DAY"] == day_num]
                    if stop_num <= day_data.shape[0]:
                        cust_name = (
                            day_data.iloc[stop_num - 1]["CUST NAME"]
                            if stop_num <= day_data.shape[0]
                            else ""
                        )
                        html_content += f"<td>{stop_num}. {cust_name}</td>"
                    else:
                        # Empty cell for days with fewer stops or if the day does not exist
                        html_content += "<td></td>"
                html_content += "</tr>"
            html_content += "</table><br>"  # Close the table and add a space

    html_content += "</body></html>"

    # Writing the HTML content to a file
    with open("routes_summary.html", "w") as html_file:
        html_file.write(html_content)
    m = HTML(html_content)
    # display(m)
    return m


def print_route_statistics(dataset):
    # Group the data by 'NEW RT' and calculate the required statistics
    route_stats = (
        dataset.groupby("NEW RT")
        .agg(
            total_BIG_5_AVG=("BIG 5 AVG", "sum"),
            total_COMP_AVG=("COMP AVG", "sum"),
            number_of_NEW_DAY=("NEW DAY", "nunique"),
            number_of_NEW_STOP=("NEW STOP", "count"),
            average_NEW_DIST=("NEW DIST", "mean"),
        )
        .reset_index()
    )

    # Calculate Average Stops per day for each route
    route_stats["average_stops_per_day"] = (
        route_stats["number_of_NEW_STOP"] / route_stats["number_of_NEW_DAY"]
    )

    # Print the statistics in a user-friendly way
    for _, row in route_stats.iterrows():
        print(f"Route {int(row['NEW RT'])}:", ("***" * 35))
        print(
            f"  BIG 5:\t${row['total_BIG_5_AVG']:,.2f}\tRT Days:\t{int(row['number_of_NEW_DAY'])}\tStops/Day:\t{row['average_stops_per_day']:.1f}\tBig 5/Stop:\t${row['total_BIG_5_AVG']/row['number_of_NEW_STOP']:,.2f}"
        )
        print(
            f"  COMP: \t${row['total_COMP_AVG']:,.2f}\t# of Stops:\t{int(row['number_of_NEW_STOP'])}\tDist/Day:\t{row['average_NEW_DIST']:.1f}\tComp/Stop:\t${row['total_COMP_AVG']/row['number_of_NEW_STOP']:,.2f}\n"
        )


def autosize_columns(worksheet):
    for column in worksheet.columns:
        max_length = 0
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = max_length + 21
        worksheet.column_dimensions[cell.column_letter].width = adjusted_width
    return worksheet


def create_excel_by_routes(dataset):
    # now create the

    # Create a new Excel workbook and remove the default sheet
    wb = Workbook()
    wb.remove(wb.active)

    # Get unique routes from the dataset
    unique_routes = dataset["NEW RT"].unique()

    # Iterate over each unique route
    for route in unique_routes:
        # Create a new sheet for this route
        ws = wb.create_sheet(title=f"Route {route}")

        # Subset the data for the current route
        subset_data = dataset[dataset["NEW RT"] == route].sort_values(
            ["NEW DAY", "NEW STOP"]
        )

        # Find the maximum number of days for this route to define how many headers we need
        max_day = subset_data["NEW DAY"].max()
        # Calculate the number of header groups (Days 1-5, 6-10, etc.)
        num_groups = ceil(max_day / 5)

        # Set the initial row
        start_row = 1

        # Iterate over each group of days
        for group in range(num_groups):
            # Calculate the range of days for this group
            day_start = group * 5 + 1
            day_end = min((group + 1) * 5, max_day)

            # Write the day headers for this group
            for day_num in range(day_start, day_end + 1):
                col = day_num - group * 5
                ws.cell(row=start_row, column=col, value=f"Day {day_num}")

            # Populate the customer names under each day
            for day_num in range(day_start, day_end + 1):
                # Filter the subset for this day
                day_data = subset_data[subset_data["NEW DAY"] == day_num]
                # Write each customer name in this day's column
                row_num = start_row + 1  # Starting row for customer names
                for _, row in day_data.iterrows():
                    col = day_num - group * 5  # Adjust column based on group
                    ws.cell(row=row_num, column=col, value=row["CUST NAME"])
                    row_num += 1  # Move to the next row for the next customer

            # Update the starting row for the next group of days
            if not day_data.empty:
                # Add extra space before the next group
                start_row += len(day_data) + 2
        ws = autosize_columns(ws)

    # Save the workbook
    wb.save("routes_by_days.xlsx")


def calculate_avg_diff(dataset):
    avg_diff = 0
    for i in range(len(dataset)):
        avg_diff += abs(dataset["DAY"].iloc[i] - dataset["NEW DAY"].iloc[i])
    avg_diff /= len(dataset)
    return avg_diff


def order_similar_to_prev(dataset):
    # Step 1: Calculate the average 'DAY' for each 'NEW RT' and 'NEW DAY' combination
    grouped_data = dataset.groupby(["NEW RT", "NEW DAY"])[
        "DAY"].mean().reset_index()
    grouped_data = grouped_data.rename(columns={"DAY": "AVERAGE_DAY"})

    # Step 2: Sort these averages for each 'NEW RT' and reassign 'NEW DAY'
    grouped_data.sort_values(["NEW RT", "AVERAGE_DAY"], inplace=True)
    grouped_data["NEW_DAY_SORTED"] = (
        grouped_data.groupby("NEW RT").cumcount() + 1
    )  # Start from 1

    # Merge the sorted 'NEW DAY' back to the original dataset
    dataset = dataset.merge(
        grouped_data[["NEW RT", "NEW DAY", "NEW_DAY_SORTED"]],
        on=["NEW RT", "NEW DAY"],
        how="left",
    )
    dataset["NEW DAY"] = dataset["NEW_DAY_SORTED"]
    dataset.drop("NEW_DAY_SORTED", axis=1, inplace=True)
    return dataset


def print_starts_ends(data, dataset):
    for start in data["starts"]:
        print(dataset.iloc[start]["CUST NAME"])
    for endd in data["ends"]:
        print(dataset.iloc[endd]["CUST NAME"])


def best_tracker(solution, filename):
    current_obj = solution.ObjectiveValue()
    try:
        with open(filename, "r") as f:
            lines = f.readlines()
            if len(lines) >= 2:
                prev_obj = int(lines[-1])
                best_obj = int(lines[-2])
            else:
                prev_obj = current_obj
                best_obj = current_obj
    except FileNotFoundError:
        prev_obj = current_obj
        best_obj = current_obj

    # Create a table object
    table = Table(title="Best Tracker", show_lines=True)

    # Add columns to the table
    table.add_column("Solution", style="magenta")
    table.add_column("Distance", style="cyan")
    scurrent_obj = "{:,}".format(current_obj)
    sbest_obj = "{:,}".format(best_obj)
    sprev_obj = "{:,}".format(prev_obj)

    # Add rows to the table
    table.add_row("Current", scurrent_obj)
    table.add_row("Best", sbest_obj)
    table.add_row("Last Run", sprev_obj)

    if best_obj > current_obj:
        improvement = best_obj - current_obj
        best_obj = current_obj
        sbest_obj = "{:,}".format(best_obj)
        simprovement = "{:,}".format(improvement)

        table.add_row("WINNER!", sbest_obj)
        table.add_row("IMPROVEMENT AMOUNT", str(simprovement))

    else:
        if prev_obj > current_obj:
            improvement = prev_obj - current_obj
            simprovement = "{:,}".format(improvement)
            table.add_row("IMPROVEMENT AMOUNT", str(simprovement))

        else:
            deterioration = current_obj - prev_obj
            sdeterioration = "{:,}".format(deterioration)

            table.add_row("WORSE BY", str(sdeterioration))
    print(table)

    # After processing, update the file with the current and new best solutions
    with open(filename, "w") as f:
        f.write(f"{best_obj}\n{current_obj}\n")

    return prev_obj, best_obj


def set_days_for_routes(
    dataset, min_stops=6, max_stops=10, distance_threshold=80, day_factor=2
):
    # Get unique routes from the dataset
    unique_routes = dataset["NEW RT"].unique()

    # Iterate over each unique route
    for route in unique_routes:
        # Subset the data for the current route, sorted by NEW STOP to ensure order
        subset_data = dataset[dataset["NEW RT"]
                              == route].sort_values("NEW STOP")

        # Initialize variables for splitting into days
        current_day = 1
        current_stop_in_day = 1
        cumulative_distance = 0  # Continue tracking but not using for decision
        stops_in_current_day = []

        # Iterate over rows in the sorted subset data
        for i, row in subset_data.iterrows():
            # Add the stop to the current day's list
            stops_in_current_day.append(i)
            # Update the cumulative distance but not used for decision
            cumulative_distance += row["NEW DIST"]

            # Check for splitting conditions based on stop count, individual distances, and cluster threshold
            if len(stops_in_current_day) >= min_stops and (
                len(stops_in_current_day) >= max_stops
                or row["NEW DIST"] > distance_threshold
                or cumulative_distance > day_factor * distance_threshold
            ):  # Added cluster threshold check for splitting

                # Update the NEW DAY and NEW STOP values in the original dataset
                for stop_index in stops_in_current_day:
                    dataset.at[stop_index, "NEW DAY"] = current_day
                    dataset.at[stop_index, "NEW STOP"] = current_stop_in_day
                    current_stop_in_day += (
                        1  # Increment the stop number for the current day
                    )

                # Reset variables for the new day
                current_day += 1
                stops_in_current_day = []
                cumulative_distance = 0  # Reset but keep for tracking
                current_stop_in_day = 1

        # Check if there are remaining stops for the last day
        if stops_in_current_day:
            for stop_index in stops_in_current_day:
                dataset.at[stop_index, "NEW DAY"] = current_day
                dataset.at[stop_index, "NEW STOP"] = current_stop_in_day
                current_stop_in_day += 1

    # Return the updated dataset
    return dataset


def print_solution(data, manager, routing, solution, dataset):
    total_distance = 0  # Total distance covered by all the vehicles
    total_load = 0  # Total load carried by all the vehicles
    total_load2 = 0  # Total load carried by all the vehicles
    previous_index = routing.Start(0)
    new_vehicle_assignment = (
        {}
    )  # Initialize a new dictionary to track vehicle assignments

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route_distance = 0  # Distance covered by the current vehicle
        route_load = 0  # Load carried by the current vehicle
        route_load2 = 0  # Load carried by the current vehicle
        route_count = 0  # Count of stops made by the current vehicle
        while not routing.IsEnd(index):
            route_count += 1
            node_index = manager.IndexToNode(index)  #
            route_load += data["demands"][node_index]
            route_load2 += data["demands2"][node_index]
            new_vehicle_assignment[node_index] = (
                # Assign vehicle to new_vehicle_assignment instead of data['Vehicle']
                vehicle_id
            )
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        end_node_index = manager.IndexToNode(
            index
        )  # Convert the final routing index to your node index
        # Since 'index' at this point should be the end node for the current vehicle
        # Assign this end node to the corresponding vehicle
        new_vehicle_assignment[end_node_index] = (
            vehicle_id  # Assign the route to the end node
        )
        route_count += 1
        route_load += data["demands"][node_index]
        route_load2 += data["demands2"][node_index]
        new_vehicle_assignment[node_index] = (
            # Assign vehicle to new_vehicle_assignment instead of data['Vehicle']
            vehicle_id
        )

        print(
            "Route {} has {} stops ({})\nCompanion: ${:,.0f} ({})    Big 5: ${:,.0f} ({})\nDistance: {}m".format(
                vehicle_id + 1,
                route_count,
                data["vehicle_capacities3"][vehicle_id],
                route_load,
                data["vehicle_capacities"][vehicle_id],
                route_load2,
                data["vehicle_capacities2"][vehicle_id],
                route_distance,
            )
        )

        total_distance += route_distance
        total_load += (
            route_load  # Add the load carried by the vehicle to the total load
        )
        total_load2 += (
            route_load2  # Add the load carried by the vehicle to the total load
        )
    print("Total Distance of all routes: {}m".format(total_distance))
    # Assign the values from new_vehicle_assignment to dataset['NEW RT']
    # Convert new_vehicle_assignment dictionary to a pandas Series first, then add 1 to align with # At the end of your function, after completing assignments for all vehicles
    dataset["NEW RT"] = pd.Series(new_vehicle_assignment).map(
        lambda x: x + 1
    )  # Convert and correct route indices as needed

    return dataset


def assign_new_rts(data, manager, routing, solution, dataset):
    new_vehicle_assignment = {}
    new_stop_assignment = {}
    new_dist_assignment = {}

    for vehicle_id in range(data["num_vehicles"]):
        # print(f"Vehicle {vehicle_id + 1}:")
        index = solution.Value(routing.NextVar(routing.Start(vehicle_id)))
        stop = 1
        # Start index for this vehicle
        previous_index = routing.Start(vehicle_id)

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)

            # Assignments for current stop
            new_vehicle_assignment[node_index] = vehicle_id
            new_stop_assignment[node_index] = int(stop)
            # print(f"Stop {stop}: {dataset.iloc[node_index]['CUST NAME']}")
            # print(f"Index: {index}, Node Index: {node_index}, Stop: {stop}, Previous Index: {previous_index}")

            # Update distance if not at the start node
            if not routing.IsStart(previous_index):
                # Ensure the previous index and current index are within bounds of the distance matrix
                if previous_index < len(data["distance_matrix"]) and index < len(data["distance_matrix"]):
                    new_dist_value = data["distance_matrix"][previous_index][index]
                    new_dist_assignment[node_index] = new_dist_value

            # Update for the next iteration
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            stop += 1

        # Handle the final connection back to the depot, if necessary
        if not routing.IsStart(previous_index) and routing.IsEnd(index):
            # End node reached, assign default values or perform necessary finalization
            # Using previous_index as end node
            end_node_index = manager.IndexToNode(previous_index)
            new_vehicle_assignment[end_node_index] = vehicle_id
            new_stop_assignment[end_node_index] = stop
            # No distance back to depot or handle appropriately
            new_dist_assignment[end_node_index] = 0

    # Update the dataset based on assignments
    dataset["NEW RT"] = pd.Series(new_vehicle_assignment).map(lambda x: x + 1)
    dataset["NEW DAY"] = pd.Series(new_vehicle_assignment).map(lambda x: x + 1)
    dataset["NEW DIST"] = pd.Series(new_dist_assignment)
    dataset["NEW STOP"] = pd.Series(new_stop_assignment)
    dataset["NEW DIST"] = dataset["NEW DIST"].fillna(0)
    dataset["NEW STOP"] = dataset["NEW STOP"].fillna(0)
    dataset["NEW STOP"] = dataset["NEW STOP"].astype(int)
    dataset["NEW DAY"] = dataset["NEW DAY"].fillna(0)
    dataset["NEW DAY"] = dataset["NEW DAY"].astype(int)
    return dataset


def create_map_with_accessible_tooltips(dataset):
    dataset = dataset.sort_values(["NEW RT", "NEW DAY"])

    m = folium.Map(location=[39.7, -76.5], zoom_start=9, max_zoom=20)

    vehicles = dataset["NEW RT"].unique()
    colors = list(mcolors.CSS4_COLORS.keys())[13: 13 + len(vehicles)]

    color_map = dict(zip(vehicles, colors))

    for vehicle in vehicles:
        vehicle_data = dataset[dataset["NEW RT"] == vehicle]
        locations = vehicle_data[["Latitude", "Longitude"]].values

        route_group = FeatureGroup(name=f"Route {vehicle}")

        # Group data by route-day to compute hulls separately
        for (route, day), day_data in vehicle_data.groupby(["NEW RT", "NEW DAY"]):
            locations = day_data[["Latitude", "Longitude"]].values
            # print(f"Locations for route {route} day {day}:\n{locations}")
            if len(locations) > 2:
                try:

                    hull = ConvexHull(locations, qhull_options="QJ")
                    hull_points = locations[hull.vertices]
                    # print(f"Hull points for route {route} day {day}:\n{hull_points}")
                    # input("Press Enter to continue...")
                    folium.Polygon(
                        locations=hull_points.tolist(),
                        color=color_map[vehicle],
                        fill=True,
                        fill_color=color_map[vehicle],
                        fill_opacity=0.3,
                        weight=2,
                    ).add_to(route_group)
                except Exception as e:
                    print(
                        f"Could not create a convex hull for vehicle {vehicle}: {e}")

                # Markers: (Same as before)
                for index, row in day_data.iterrows():
                    tooltip_html = f"""
                    Route: {vehicle}, Day: {row['NEW DAY']}<br>
                    Customer: {row['CUST NAME']}<br>
                    Address: {row['ADDRESS']}<br>
                    Sales: ${row['Machine']:,.2f}<br>
                    Companion: ${row['Companion']:,.2f}
                    """
                    folium.CircleMarker(
                        location=[row["Latitude"], row["Longitude"]],
                        tooltip=tooltip_html,
                        color=color_map[vehicle],
                        fill=True,
                        fill_color=color_map[vehicle],
                        radius=5,
                    ).add_to(route_group)

        route_group.add_to(m)

    Fullscreen().add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)

    # create map file name with time and date
    now = datetime.datetime.now()
    map_file = f"./data/maps/nmap_with_accessible_tooltips {now.day}-{now.month}-{now.year} {now.hour}{now.minute}.html"
    m.save(map_file)
    map_file = f"./data/nmaaap_with_accessible_tooltips.html"
    m.save(map_file)
    folium.LayerControl(collapsed=False).add_to(m)
    # display(m)
    return m


def swap_route_days(dataset, rt1, rt2, day1, day2):
    """
    Swap all stops between two route-day combinations within the dataset, including changing their route numbers.

    Parameters:
        dataset (pd.DataFrame): The dataset containing route information.
        rt1, rt2 (int): The route numbers involved in the swap.
        day1, day2 (int): The day numbers to be swapped between the routes.
    """
    # Create a copy of the dataset to avoid modifying the original
    updated_dataset = dataset.copy()

    # Identify the index for each route-day combination
    index_rt_day_1 = dataset[(dataset['NEW RT'] == rt1) & (
        dataset['NEW DAY'] == day1)].index
    index_rt_day_2 = dataset[(dataset['NEW RT'] == rt2) & (
        dataset['NEW DAY'] == day2)].index

    # Check if we have records to swap to avoid errors
    if not index_rt_day_1.empty and not index_rt_day_2.empty:
        # Swap the route numbers and days between the selected route-day combinations
        updated_dataset.loc[index_rt_day_1, 'NEW RT'] = rt2
        updated_dataset.loc[index_rt_day_2, 'NEW RT'] = rt1
        updated_dataset.loc[index_rt_day_1, 'NEW DAY'] = day2
        updated_dataset.loc[index_rt_day_2, 'NEW DAY'] = day1
    else:
        print("One of the routes/days to be swapped does not exist in the dataset.")

    return updated_dataset


def move_route_stops(dataset, from_rt, from_day, to_rt, to_day, list_of_stops):
    """
    Move specified stops from one route-day combination to another, 
    appending them to the end of the new route-day.

    Parameters:
        dataset (pd.DataFrame): The dataset containing route information.
        from_rt, to_rt (int): The original and destination route numbers.
        from_day, to_day (int): The original and destination day numbers.
        list_of_stops (list): List of stops to be moved.
    """
    # Create a copy of the dataset to avoid modifying the original
    updated_dataset = dataset.copy()

    # Filter for the stops to be moved
    stops_to_move = updated_dataset[(updated_dataset['NEW RT'] == from_rt) &
                                    (updated_dataset['NEW DAY'] == from_day) &
                                    (updated_dataset['STOP'].isin(list_of_stops))]

    # Check if there are stops to move
    if not stops_to_move.empty:
        # Update 'NEW RT' and 'NEW DAY' for these stops
        updated_dataset.loc[stops_to_move.index, 'NEW RT'] = to_rt
        updated_dataset.loc[stops_to_move.index, 'NEW DAY'] = to_day

        # Assuming 'NEW STOP' indicates the ordering within the day
        # Find the max 'NEW STOP' in the destination route-day to append these stops at the end
        max_stop_in_destination = updated_dataset[(updated_dataset['NEW RT'] == to_rt) &
                                                  (updated_dataset['NEW DAY'] == to_day)]['NEW STOP'].max()

        # Update 'NEW STOP' for the moved stops, appending them to the end
        # Start from 1 to add to the max
        for i, stop in enumerate(sorted(list_of_stops), start=1):
            updated_dataset.loc[(updated_dataset['STOP'] == stop) &
                                (updated_dataset['NEW RT'] == to_rt) &
                                (updated_dataset['NEW DAY'] == to_day), 'NEW STOP'] = max_stop_in_destination + i
    else:
        print("No matching stops found to move.")

    return updated_dataset


def haversine_vectorized(latitudes, longitudes, depot_index=0, unit="kilometers"):
    """
    Vectorized version of the Haversine formula to calculate the pairwise distance matrix between two sets of geographical points.
    Additionally incorporates custom logic to simulate the 'distance_callback' adjustments.
    """
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(latitudes)
    lon_rad = np.radians(longitudes)

    # Expand lat_rad and lon_rad into 2D arrays for broadcasting
    lat_rad_matrix = np.expand_dims(lat_rad, axis=0)
    lon_rad_matrix = np.expand_dims(lon_rad, axis=0)

    # Compute pairwise differences
    dlat = lat_rad_matrix - lat_rad_matrix.T
    dlon = lon_rad_matrix - lon_rad_matrix.T

    # Haversine formula
    a = (
        np.sin(dlat / 2.0) ** 2
        + np.cos(lat_rad_matrix) * np.cos(lat_rad_matrix.T) *
        np.sin(dlon / 2.0) ** 2
    )
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance_matrix = EARTH_RADIUS_KM * c  # Distance in kilometers

    # Convert distance according to the unit
    conversion_factors = {
        "miles": 0.621371,
        "kilometers": 1,
        "nautical miles": 0.539957,
    }
    if unit in conversion_factors:
        distance_matrix *= conversion_factors[unit]
    else:
        raise ValueError("Invalid unit")

    # Custom logic adjustment based on distance_callback rules
    # Adding additional cost based on depot distances
    for from_node in range(len(latitudes)):
        from_depot_cost = distance_matrix[depot_index][from_node]
        for to_node in range(len(latitudes)):
            to_depot_cost = distance_matrix[depot_index][to_node]
            if to_depot_cost > from_depot_cost:
                distance_matrix[from_node][to_node] += 11
    # set the first row and column to 0
    distance_matrix[0] = 0
    distance_matrix[:, 0] = 0
    
    return distance_matrix


def create_distance_matrix(dataset):
    # Example usage
    latitudes = dataset["Latitude"].to_numpy()
    longitudes = dataset["Longitude"].to_numpy()
    # time the function
    import time

    start = time.time()
    # print("Creating distance matrix...")

    distance_matrix = haversine_vectorized(
        latitudes, longitudes, unit="kilometers")
    # calculate the time taken
    end = time.time()
    # print(f"Time taken: {end - start} seconds")
    # Convert the distance matrix to a DataFrame if needed
    distance_matrix = pd.DataFrame(
        distance_matrix, index=dataset.index, columns=dataset.index
    )
    distance_matrix *= 10
    distance_matrix += 0.9999
    # multiply all values of distance_matrix by 2 and convert to int
    distance_matrix = distance_matrix.astype(int)

    # Create a boolean mask where all diagonal elements are False and others are True
    mask = ~np.eye(distance_matrix.shape[0], dtype=bool)

    # Add 1 to all elements that are not on the diagonal (mask is True) and are currently 0
    distance_matrix = distance_matrix.mask(
        mask & (distance_matrix == 0), distance_matrix + 1
    )
    distance_matrix = distance_matrix.astype(int)

    distance_matrix.to_csv("distance_matrix.csv", index=False)
    # print("Distance matrix created.")
    return distance_matrix


def find_unique_name_indexes(data_frame, custom_names):
    """
    Finds the indexes of unique custom names in a DataFrame's 'CUST NAME' column.

    Args:
        data_frame: A Pandas DataFrame.
        custom_names: A list of unique custom names to search for.

    Returns:
        A list of the indexes of the custom names in the DataFrame. If a name
        is not found, its corresponding index in the result list will be None.
    """
    result = []
    for name in custom_names:
        mask = data_frame['CUST NAME'] == name
        index = data_frame.loc[mask].index.tolist()  # Get index as a list
        result.append(index[0] if index else None)  # First index or None

    return result


def best_tracker_with_params(solution, filename, search_parameters):
    current_obj = solution.ObjectiveValue()
    try:
        with open(filename, "r") as f:
            lines = f.readlines()
            if len(lines) >= 2:
                prev_obj = int(lines[-1])
                best_obj = int(lines[-2])
            else:
                prev_obj = current_obj
                best_obj = current_obj
    except FileNotFoundError:
        prev_obj = current_obj
        best_obj = current_obj

    # Create a table object for best tracker
    best_tracker_table = Table(title="Progress Tracker", show_lines=True)
    best_tracker_table.add_column("Solution", style="magenta")
    best_tracker_table.add_column("Value", style="cyan")
    best_tracker_table.add_row("[b]Current", "[b]{:,}".format(current_obj))

    best_tracker_table.add_row("Best", "{:,}".format(best_obj))
    best_tracker_table.add_row("Last Run", "{:,}".format(prev_obj))

    if best_obj > current_obj:
        improvement = best_obj - current_obj
        best_obj = current_obj
        best_tracker_table.add_row(
            "[b]WINNER![/b]", "[b]{:,}".format(best_obj))
        best_tracker_table.add_row(
            "[b]IMPROVEMENT AMOUNT", "[b]{:,}".format(improvement))
    else:
        if prev_obj > current_obj:
            improvement = prev_obj - current_obj
            best_tracker_table.add_row(
                "[b]IMPROVEMENT AMOUNT", "[b]{:,}".format(improvement))
        else:
            deterioration = current_obj - prev_obj
            best_tracker_table.add_row(
                "[b]WORSE BY", "[b]{:,}".format(deterioration))
    best_tracker_table.add_row("Explore Coefficient", "{:,}".format(
        search_parameters.multi_armed_bandit_compound_operator_exploration_coefficient))
    best_tracker_table.add_row("Memory Coefficient", "{:1f}".format(
        search_parameters.multi_armed_bandit_compound_operator_memory_coefficient))
    best_tracker_table.add_row("Solve Time (seconds)", str(
        search_parameters.time_limit.seconds))
    # After processing, update the file with the current and new best solutions
    with open(filename, "w") as f:
        f.write(f"{best_obj}\n{current_obj}\n")
    return best_tracker_table


def process_solution(data, manager, routing, solution, dataset):
    total_distance = 0
    total_load = 0
    total_load2 = 0
    route_details = []
    new_vehicle_assignment = {}  # This will map node index to vehicle ID

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route_load2 = 0
        route_count = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            route_load2 += data["demands2"][node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            route_count += 1
            # Assign the vehicle to the nodes it visits
            # +1 for 1-indexed vehicle IDs
            new_vehicle_assignment[node_index] = vehicle_id + 1

        # Add the end node for each vehicle
        new_vehicle_assignment[manager.IndexToNode(
            index)] = vehicle_id + 1  # Assign end node

        total_distance += route_distance
        total_load += route_load
        total_load2 += route_load2

        # Collect route details for return
        route_details.append({
            'vehicle_id': vehicle_id + 1,
            'stops': route_count,
            'load': route_load,
            'load2': route_load2,
            'distance': route_distance
        })

    # Apply new vehicle assignments to dataset
    # Ensure every index/node is in new_vehicle_assignment, then update 'NEW RT'
    dataset['NEW RT'] = dataset.index.map(
        lambda idx: new_vehicle_assignment.get(idx, np.nan))
    dataset['NEW RT'] = dataset['NEW RT'].fillna(
        method='ffill').fillna(method='bfill')

    # Compile overall results
    results = {
        'dataset': dataset,
        'total_distance': total_distance,
        'total_load': total_load,
        'total_load2': total_load2,
        'route_details': route_details
    }
    return results


def combined_tracker_and_statistics(solution, filename, dataset, data, manager, routing, search_parameters):
    # Assuming you have other code here for setup...

    # Now call your updated best tracker with parameters

    # Assuming you have more code here for the rest of the function...
    """
    Combines tracking of solution quality, route statistics, and solution specifics 
    into a comprehensive overview using Rich tables for readability and user-friendliness.

    Args:
        solution: The routing solution object.
        filename: Name of the file used to track the best and previous objectives.
        dataset: The dataset containing route information.
        data: The data dictionary used in the routing problem.
        manager: The index manager for the routing problem.
        routing: The routing model instance.
    """

    # returns a table of the best tracker
    # sleep(1)
    best_tracker_table = best_tracker_with_params(
        solution, filename, search_parameters)

    # Generate route statistics and integrate them into the dataset
    route_data = process_solution(data, manager, routing, solution, dataset)

    # Prepare the tables for display
    stats_table = Table(title="Route Statistics", show_lines=True)
    # stats_table.add_column("Route ID", style="magenta")
    stats_table.add_column("Total BIG 5 AVG", style="cyan", max_width=10)
    stats_table.add_column("Total COMP AVG", style="green", max_width=7)
    stats_table.add_column("Number of Days", style="cyan", max_width=6)
    stats_table.add_column("Number of Stops", style="green", max_width=6)
    # stats_table.add_column("Average Stops/Day", style="red",max_width=7)
    stats_table.add_column("Ttl Distance", style="cyan", max_width=8)
    # stats_table.add_column("BIG 5/Stop", style="blue",max_width=7)
    stats_table.add_column("COMP / Stop", style="green", max_width=7)

    # Populate route statistics table
    route_stats = generate_route_statistics(dataset)
    for _, row in route_stats.iterrows():
        stats_table.add_row(
            # f"{int(row['NEW RT'])}",
            f"${int(row['total_BIG_5_AVG']):,}",
            f"${int(row['total_COMP_AVG']):,}",
            f"{int(row['number_of_NEW_DAY'])}",
            f"{int(row['number_of_NEW_STOP'])}",
            # f"{row['average_stops_per_day']:.1f}",
            f"{int(row['average_NEW_DIST']):,}",
            # f"${row['total_BIG_5_AVG']/row['number_of_NEW_STOP']:,.2f}",
            f"${int(row['total_COMP_AVG']/row['number_of_NEW_STOP'])}"
        )
    console = Console()

    panel1 = Panel(best_tracker_table, title="Best Tracker")
    panel2 = Panel(stats_table)
    tables_side_by_side = Columns([best_tracker_table, stats_table])
    # Print the tables
    print(tables_side_by_side)
    return dataset  # Return the modified dataset for potential further use


def generate_route_statistics(dataset):
    # Group the data by 'NEW RT' and calculate the required statistics
    route_stats = (
        dataset.groupby("NEW RT")
        .agg(
            total_BIG_5_AVG=("BIG 5 AVG", "sum"),
            total_COMP_AVG=("COMP AVG", "sum"),
            number_of_NEW_DAY=("NEW DAY", "nunique"),
            number_of_NEW_STOP=("NEW STOP", "count"),
            average_NEW_DIST=("NEW DIST", "sum"),
        )
        .reset_index()
    )

    # Calculate Average Stops per day for each route
    route_stats["average_stops_per_day"] = (
        route_stats["number_of_NEW_STOP"] / route_stats["number_of_NEW_DAY"]
    )
    return route_stats


def add_lat_long_to_df(dataset):
    api_key = "8dad1cc087374a6ba988e6371e45911b"
    # Update Coordinates
    # Check for existing coordinates file
    if os.path.exists("./coordinates.csv"):
        # print("Coordinates file exists.")
        coordinates_df = (
            pd.read_csv("./coordinates.csv")
            .drop_duplicates(subset="CUST")
            .reset_index(drop=True)
        )
        coordinates_df["CUST"] = coordinates_df["CUST"].astype(
            str
        )  # Convert the 'CUST' column to string
        print(coordinates_df["CUST"])
    else:
        print("Creating new coordinates DataFrame...")
        coordinates_df = pd.DataFrame(
            columns=["CUST", "Latitude", "Longitude"])
    print("Updating coordinates...")
    # Iterate through the dataset
    for i in tqdm(range(len(dataset))):
        cust = str(dataset.at[i, "CUST"])  # Ensure CUST is treated as a string

        # Check if the customer is already in the coordinates DataFrame
        if cust in coordinates_df["CUST"].values:
            # print(f"Using existing coordinates for {dataset.at[i, 'CUST NAME']}")
            row = coordinates_df[coordinates_df["CUST"] == cust]
            dataset.at[i, "Latitude"] = row["Latitude"].values[0]
            dataset.at[i, "Longitude"] = row["Longitude"].values[0]
        else:
            # Construct the address
            print(f"I = {i}    Name: {dataset.at['CUST NAME']}")
            name = dataset.at[i, "CUST NAME"]
            house_number = dataset.at[i, "HOUSE NUMBER"]
            address = dataset.at[i, "ADDRESS"]
            city = dataset.at[i, "CITY"]
            short_zip = str(dataset.at[i, "ZIP"])[:5]
            print(
                f"Name: {name}, House Number: {house_number}, Address: {address}, City: {city}, ZIP: {short_zip}")
            print(f"Quote Plus Name: {quote_plus(name)}")
            print(f"Quote Plus House Number: {quote_plus(house_number)}")
            print(f"Quote Plus Address: {quote_plus(address)}")
            print(f"Quote Plus City: {quote_plus(city)}")
            print(f"Quote Plus ZIP: {quote_plus(short_zip)}")

            # Encode components for URL
            url = f"https://api.geoapify.com/v1/geocode/search?name={quote_plus(name)}&housenumber={quote_plus(house_number)}&street={quote_plus(address)}&city={quote_plus(city)}&postcode={short_zip}&format=json&apiKey={api_key}"
            print(url)
            # Make the request
            resp = requests.get(url, headers={"Accept": "application/json"})

            if resp.status_code == 200:
                resp_query = resp.json()
                if resp_query["results"]:
                    lat, lon = (
                        resp_query["results"][0]["lat"],
                        resp_query["results"][0]["lon"],
                    )
                    dataset.at[i, "Latitude"] = lat
                    dataset.at[i, "Longitude"] = lon
                    # Append new coordinates to coordinates_df and the CSV file
                    new_row = pd.DataFrame(
                        [[cust, lat, lon]], columns=[
                            "CUST", "Latitude", "Longitude"]
                    )
                    coordinates_df = pd.concat(
                        [coordinates_df, new_row], ignore_index=True)
                    new_row.to_csv("coordinates.csv", mode="a",
                                   header=False, index=False)
                    print(f"Added coordinates for {name}")
                else:
                    print(f"No results found for {name}")
            else:
                print(
                    f"Failed to get coordinates for {name}: HTTP {resp.status_code}")
    dataset.to_csv("updated_dataset.csv", index=False)
    # Save the updated coordinates DataFrame
    coordinates_df.to_csv("coordinates.csv", index=False, mode="w")
    return dataset

    # save dataset to csv


# Constants
EARTH_RADIUS_KM = 6371  # Earth's radius in kilometers
R = 6371.0

# Setup Dataset
file_path = "./data/uploads/"
file_name = "P4reroute.xlsx"
def load_depot():
    depot = pd.read_csv(filepath_or_buffer="depot.csv")
    dataset = pd.concat([depot, dataset]).reset_index(drop=True)

# load reroute.xls into dataset, set the column headers to the first row of the dataset and drop the second row
dataset = pd.read_excel(file_path + file_name)
dataset = dataset.iloc[1:, 0:]
dataset = dataset.reset_index(drop=True)
# if dataset does not have a column named 'Latitude' then create it
if "Latitude" not in dataset.columns:
    dataset["Latitude"] = np.nan
# if dataset does not have a column named 'Longitude' then create it
if "Longitude" not in dataset.columns:
    dataset["Longitude"] = np.nan

try:
    dataset["Machine"] = dataset["BIG 5 AVG"]
    dataset["Companion"] = dataset["COMP AVG"]
except Exception as e:
    print(e)

# Assuming 'dataset' is your DataFrame
# Use regular expressions to remove any non-numeric characters from the 'NEW DAY' column
dataset['DAY'] = dataset['DAY'].replace(
    to_replace=[r'[^\d]+'], value='', regex=True)

# Convert the cleaned 'NEW DAY' values from strings to integers
# Note: This will convert empty strings to NaN, which you might need to handle based on your use case
dataset['DAY'] = pd.to_numeric(
    dataset['DAY'], errors='coerce').fillna(0).astype(int)
dataset["RT"] = dataset["RT"].astype(int)
# split the 'ADDRESS' column by comma and keep only the first part
dataset["ADDRESS"] = dataset["ADDRESS"].str.split(",").str[0]

dataset = add_lat_long_to_df(dataset)


distance_matrix = create_distance_matrix(dataset)
latlong_df = dataset[["Latitude", "Longitude"]]




dataset['NEW DAY'] = dataset['NEW DAY'].replace(
    to_replace=[r'[^\d]+'], value='', regex=True)

# Convert the cleaned 'NEW DAY' values from strings to integers
# Note: This will convert empty strings to NaN, which you might need to handle based on your use case
dataset['NEW DAY'] = pd.to_numeric(
    dataset['NEW DAY'], errors='coerce').fillna(0).astype(int)
dataset["NEW RT"] = dataset["NEW RT"].fillna(0).astype(int)
dataset['NEW STOP'] = dataset['NEW STOP'].fillna(0).astype(int)
# Create a 'Stop ID' column using the row index
dataset['Stop ID'] = dataset.index


0        6501024
1       65050465
2         650982
3         650437
4        6501054
         ...    
541    216800026
542    236500026
543            1
544    246500008
545         8912
Name: CUST, Length: 546, dtype: object

Updating coordinates...

100%|██████████| 353/353 [00:00<00:00, 2069.23it/s]


ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
def set_search_parameters(
    search_parameters,
    solve_minutes=1,
    explore_coefficient=99999999,
    memory_coefficient=0.09,
    log_search=False,
    seconds_per_iter=10,
    solution_limit=999999,
):
    """
    Set the search parameters for the routing problem.

    Args:
        search_parameters: The search parameters object.
        solve_minutes: The number of minutes to solve the problem (default: 1).
        explore_coefficient: The exploration coefficient for the multi-armed bandit compound operator (default: 99999999).
        memory_coefficient: The memory coefficient for the multi-armed bandit compound operator (default: 0.09).

    Returns:
        The updated search parameters object.
    """
    F = pywrapcp.BOOL_FALSE
    T = pywrapcp.BOOL_TRUE

    search_parameters.local_search_operators.use_cross = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_exchange = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_cross_exchange = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_opt = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_or_opt = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_lin_kernighan = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_two_opt = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE

    search_parameters.local_search_operators.use_exchange_subtrip = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_exchange_pair = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_inactive = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_light_relocate_pair = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate_subtrip = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate_and_make_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_extended_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_node_pair_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_chain_inactive = pywrapcp.BOOL_TRUE

    search_parameters.local_search_operators.use_relocate_expensive_chain = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_inactive_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_global_cheapest_insertion_close_nodes_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_local_cheapest_insertion_close_nodes_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_global_cheapest_insertion_expensive_chain_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_local_cheapest_insertion_expensive_chain_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_global_cheapest_insertion_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_local_cheapest_insertion_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.use_multi_armed_bandit_concatenate_operators = pywrapcp.BOOL_TRUE

    search_parameters.multi_armed_bandit_compound_operator_memory_coefficient = (
        memory_coefficient)
    search_parameters.multi_armed_bandit_compound_operator_exploration_coefficient = (
        explore_coefficient)

    search_parameters.relocate_expensive_chain_num_arcs_to_consider = 90
    search_parameters.heuristic_expensive_chain_lns_num_arcs_to_consider = 99
    search_parameters.heuristic_close_nodes_lns_num_nodes = 90
    search_parameters.use_depth_first_search = pywrapcp.BOOL_TRUE
    search_parameters.solution_limit = solution_limit
    # search_parameters.time_limit.seconds = int(
    #     solve_minutes * seconds_per_iter)  # compute_time.value
    # search_parameters.lns_time_limit.nanos = 1
    # search_parameters.number_of_solutions_to_collect = 3

    search_parameters.lns_time_limit.seconds = 31
    search_parameters.use_cp_sat = pywrapcp.BOOL_TRUE
    search_parameters.fallback_to_cp_sat_size_threshold = 99
    search_parameters.cheapest_insertion_first_solution_min_neighbors = 9
    search_parameters.cheapest_insertion_first_solution_neighbors_ratio = 0.9
    # search_parameters.secondary_ls_time_limit_ratio = 0.04
    log_search = True
    search_parameters.log_search = log_search
    search_parameters.use_full_propagation = False
    # search_parameters.log_cost_scaling_factor = 0.1
    return search_parameters


# Prime the Routing Model

In [ ]:
def create_data_model(
    dataset,
    distance_matrix,
    vehicle_capacities=[],
    vehicle_capacities2=[],
    vehicle_capacities3=[],
    starts=[],
    ends=[]
):
    max_stops_per_day = 9
    max_comp_per_day = 1900
    max_big5_per_day = 3500
    data = {}
    demands = dataset["Companion"].tolist()
    demands2 = dataset["Machine"].tolist()
    data["num_vehicles"] = 50
    data["distance_matrix"] = distance_matrix.values.tolist()
    data["demands"] = [int(demand) for demand in demands]
    data["demands2"] = [int(demand) for demand in demands2]
    data["demands3"] = [1] * len(data["demands"])
    # data['vehicle_capacities']  = [int(dataset["COMP AVG"].sum() / data['num_vehicles'])] * data['num_vehicles'] # comp target max for each route
    # data['vehicle_capacities2'] = [int(dataset["BIG 5 AVG"].sum() / data['num_vehicles'])] * data['num_vehicles'] # big 5 target max for each route
    data['vehicle_capacities'] = [max_comp_per_day] * \
        data['num_vehicles']  # comp target max for each route
    data['vehicle_capacities2'] = [max_big5_per_day] * \
        data['num_vehicles']  # big 5 target max for each route
    # number of stops for each route[int(dataset["CUST"].count() / data['num_vehicles'])]
    data['vehicle_capacities3'] = [max_stops_per_day] * data['num_vehicles']
    # data["vehicle_capacities"] = [int(capacity)

    cap_factor = 3.8
    # data['vehicle_capacities'] = [int(15  * capacity) for capacity in data['vehicle_capacities']]
    # data['vehicle_capacities2'] = [int(1.2 * capacity) for capacity in data['vehicle_capacities2']]
    # data['vehicle_capacities3'] = [int(1.5 * capacity) for capacity in data['vehicle_capacities3']]
    data["depot"] = 0

    return data


In [ ]:
seconds_for_prime = 22
test_runs = 0
max_test_runs = 8
explore_prime = 0
relocate_expensive_arcs = 33
heuristic_expensive_arcs = 33
heuristic_close_nodes = 33
memory_coefficient = 0.77
max_distance = 1878
solution_limit = 2900
final_fig = [None] * solution_limit


In [ ]:
####### Click to run Solver#############################
import random

best_obj = max_distance

filenames = []


data = create_data_model(
    dataset,
    distance_matrix,
)
# print(data)
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

routing_parameters = pywrapcp.DefaultRoutingModelParameters()
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager, routing_parameters)


routes = {}
for index, row in dataset.iterrows():
    if (row["NEW RT"], row["NEW DAY"]) not in routes:
        routes[(row["NEW RT"], row["NEW DAY"])] = []
    routes[(row["NEW RT"], row["NEW DAY"])].append(index)  # Store the row index


# manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"])


def create_distance_callback(data, manager, vehicle):
    _distances = data["distance_matrix"]
    # _starts = data["starts"]
    # _ends = data["ends"]
    # Capture current value of vehicle

    def distance_callback(from_index, to_index, _vehicle=vehicle):
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        # depot_index = manager.GetDepot()
        # depot_node = manager.IndexToNode(depot_index)
        # start_node = _starts[_vehicle]
        # end_node = _ends[_vehicle]
        # _home_cost = _distances[start_node][to_node]
        # _end_cost = _distances[to_node][end_node]
        to_cost = _distances[from_node][to_node]
        # from_depot_cost = _distances[0][from_node]
        # to_depot_cost = _distances[0][to_node]
        # if (to_node == 242 or to_node == 201 or to_node == 194) and vehicle != 1:
        #     to_cost += 4100
        # if (from_node == 242 or from_node == 201 or to_node == 194) and vehicle != 1:
        #     to_cost += 4100
        # off_track_factor = ((_home_cost + _end_cost)/8) ** 2
        # _arc_cost = int((to_cost * off_track_factor) / 10000)
        # check to see if to or from index is for the depot
        # if routing.GetDepot() == from_node or routing.GetDepot() == to_node:
        # return 0
        # if from_node == manager.GetDepot() or to_node == manager.GetDepot():
        #     return 0
        # if to_cost > 20:
        #     to_cost = int((to_cost /15) ** 2)
        # else:
        #     to_cost //= 2
        # if routing.VehicleRouteConsideredVar(_vehicle) != 1:
        # # if routing.ActiveVehicleVar(_vehicle) == 1:
        #     to_cost += 350
        #     # print(_vehicle)
        # if routing.GetDepot() == from_index:
        #     to_cost = 1#
        # else:
        #     to_cost += 350

        # if routing.GetDepot() == to_index:
        #     # print(from_index)
        #     to_cost += 1# random.randint(1, 2)
        # # low_penalty = get_num_stops(routing, solution, vehicle)
        #     # print(f"from_node: {from_node} to_node: {to_node} to_index: {to_index} depot: {routing.GetDepot()}")
        # arc_cost =int(((to_cost**2)/100) - ((to_depot_cost + from_depot_cost )/9))
        # arc_cost = to_cost
        # if to_depot_cost > from_depot_cost:
        #     arc_cost = int(arc_cost + 11)

        return to_cost

    return distance_callback


rSolve = routing.solver()

for vehicle_id in range(data["num_vehicles"]):
    distance_callback = create_distance_callback(data, manager, vehicle_id)
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfVehicle(distance_callback_index, vehicle_id)


def demand_callback(from_index):
    return data["demands"][manager.IndexToNode(from_index)]


def demand_callback2(from_index):
    return data["demands2"][manager.IndexToNode(from_index)]


def demand_callback3(from_index):
    return data["demands3"][manager.IndexToNode(from_index)]


# Create and register 3 transit callbacks. 1=companion 2=big 5 and 3=1 to count a single stop
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
demand_callback_index2 = routing.RegisterUnaryTransitCallback(demand_callback2)
demand_callback_index3 = routing.RegisterUnaryTransitCallback(demand_callback3)

routing.AddDimension(distance_callback_index, 150, max_distance, True, "Distance")
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index, 100, data["vehicle_capacities"], True, "Capacity"
)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index2, 2000, data["vehicle_capacities2"], True, "Capacity2"
)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index3, 2, data["vehicle_capacities3"], True, "Capacity3"
)
# for vehicle_id in range(data["num_vehicles"]):
#     routing.SetVehicleUsedWhenEmpty(True, vehicle_id)
##################################################################
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_COST_INSERTION
)
solve_minutes = 1

m = []
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.local_search_operators.use_cross = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_relocate = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_exchange = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_cross_exchange = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_tsp_opt = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_or_opt = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_lin_kernighan = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_two_opt = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE

search_parameters.local_search_operators.use_exchange_subtrip = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_exchange_pair = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_make_inactive = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_light_relocate_pair = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_relocate_subtrip = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_relocate_and_make_active = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_extended_swap_active = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_node_pair_swap_active = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_swap_active = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_make_chain_inactive = pywrapcp.BOOL_TRUE

search_parameters.local_search_operators.use_relocate_expensive_chain = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_inactive_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_tsp_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_global_cheapest_insertion_close_nodes_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_local_cheapest_insertion_close_nodes_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_global_cheapest_insertion_expensive_chain_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_local_cheapest_insertion_expensive_chain_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_global_cheapest_insertion_path_lns = pywrapcp.BOOL_TRUE
search_parameters.local_search_operators.use_local_cheapest_insertion_path_lns = pywrapcp.BOOL_TRUE
search_parameters.use_multi_armed_bandit_concatenate_operators = pywrapcp.BOOL_TRUE
search_parameters.multi_armed_bandit_compound_operator_memory_coefficient = memory_coefficient
search_parameters.multi_armed_bandit_compound_operator_exploration_coefficient = explore_prime
search_parameters.relocate_expensive_chain_num_arcs_to_consider = relocate_expensive_arcs
search_parameters.heuristic_expensive_chain_lns_num_arcs_to_consider = heuristic_expensive_arcs
search_parameters.heuristic_close_nodes_lns_num_nodes = heuristic_close_nodes

search_parameters.time_limit.seconds = int(
    solve_minutes * seconds_for_prime
)  # compute_time.value
search_parameters.solution_limit = solution_limit
search_parameters.lns_time_limit.seconds = 19
# search_parameters.secondary_ls_time_limit_ratio = 0.05
search_parameters.log_search = True
search_parameters.use_full_propagation = False


def get_stop_id_from_index(index):
    return dataset.loc[index, "Stop ID"]


high_penalty = 50

for route in routes.values():  # Assuming routes contains lists of indices
    route_indices = route  # Ensure route is a list of integer indices
    routing.AddSoftSameVehicleConstraint(route_indices, high_penalty)

search_parameters.use_cp_sat = pywrapcp.BOOL_FALSE
search_parameters.use_generalized_cp_sat = pywrapcp.BOOL_FALSE
search_parameters.guided_local_search_lambda_coefficient = 0.9

# search_parameters.fallback_to_cp_sat_size_threshold = 90
search_parameters.cheapest_insertion_first_solution_min_neighbors = 39
search_parameters.cheapest_insertion_first_solution_neighbors_ratio = 0.9
search_parameters.sat_parameters.num_search_workers = 12
search_parameters.sat_parameters.linearization_level = 11
print("data: starting to solve")


class SolutionCallback:
    """Create a solution callback."""

    def __init__(
        self,
        manager: pywrapcp.RoutingIndexManager,
        model: pywrapcp.RoutingModel,
        limit: int,
    ):
        # We need a weak ref on the routing model to avoid a cycle.
        self._routing_manager_ref = weakref.ref(manager)
        self._routing_model_ref = weakref.ref(model)
        self._counter = 0
        self._counter_limit = limit
        self.objectives = []
        self.progress = 0
        self.accepted_neighbors = 0

    def __call__(self):
        # print(f"ok: {self._routing_model_ref.solver().SearchTrace('abc')}")
        accepted_neighbors = self._routing_model_ref().solver().AcceptedNeighbors()
        progress = int(self._routing_model_ref().solver().TopProgressPercent())
        objective = int(self._routing_model_ref().CostVar().Value())
        branches = self._routing_model_ref().solver().Branches()
        model_stamp = self._routing_model_ref().solver().Stamp()
        wall_time = self._routing_model_ref().solver().WallTime()
        self._routing_model_ref()
        if not self.objectives or objective < self.objectives[-1]:
            self.objectives.append(objective)
            # print_solution(self._routing_manager_ref(), self._routing_model_ref())
            self._counter += 1
            if self._counter % 5 == 0:
                print(
                    f"Solution #{self._counter}  found as: {objective} Progress: {progress}%  Branches: {branches} Accepted Neighbors: {accepted_neighbors} Model Stamp {model_stamp} Wall Time: {wall_time}"
                )

# Assuming 'data', 'manager', 'routing', and 'search_parameters' are already defined:
solution_callback = SolutionCallback(manager, routing, 19400)
routing.AddAtSolutionCallback(solution_callback, True)
search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE
# search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE

# Define a large penalty for empty routes


solution = routing.SolveWithParameters(search_parameters)


solution_limit += 20
print(
    f"Objective: {solution.ObjectiveValue()} Reloc Exp Arcs: {relocate_expensive_arcs} Heur Exp Arcs: {heuristic_expensive_arcs} Heur Close Nodes: {heuristic_close_nodes} Prime Time: {seconds_for_prime} Explore Prime: {explore_prime}"
)
if best_obj > solution.ObjectiveValue():
    best_obj = solution.ObjectiveValue()
    # add a row to the best settings dataframe using concat
# best_settings = pd.concat([best_settings, pd.DataFrame({"Objective Value": [solution.ObjectiveValue()], "Relocate Expensive Arcs": [relocate_expensive_arcs], "Heuristic Expensive Arcs": [heuristic_expensive_arcs], "Heuristic Close Nodes": [heuristic_close_nodes], "Seconds for Prime": [seconds_for_prime], "Explore Prime": [explore_prime]})])
print(
    f"Objective: {solution.ObjectiveValue()} Reloc Exp Arcs: {relocate_expensive_arcs} Heur Exp Arcs: {heuristic_expensive_arcs} Heur Close Nodes: {heuristic_close_nodes} Prime Time: {seconds_for_prime} Explore Prime: {explore_prime}"
)
# solution = rSolve.Solve(search_parameters)
# trash = print_solution(data, manager, routing, solution, dataset)
search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE

filename = "./data/best_tracker85.csv"
# assign new route days and stops
dataset = assign_new_rts(data, manager, routing, solution, dataset)
average_new_dist = (
    dataset.groupby(["NEW RT", "NEW DAY", "NEW STOP"])["NEW DIST"].sum().quantile(0.95)
)
# split the routes into days
dataset = set_days_for_routes(
    dataset,
    min_stops=1,
    max_stops=10,
    distance_threshold=999,  # average_new_dist,
    day_factor=12,
)
_ = combined_tracker_and_statistics(
    solution, filename, dataset, data, manager, routing, search_parameters
)
m.append(create_map_with_accessible_tooltips(dataset))
# write solution to csv
# create_excel_by_routes(dataset)
# routes_by_days_to_html(dataset)
print(solution.ObjectiveValue())
dataset.to_csv("final_dataset.csv", index=False)
# dispay the image final frame png file
fname = create_final_route_frame(dataset)  # Display the final frame
filenames.append(fname)
# final_fig_OG = (create_final_route_frame(dataset_OG)) # Display the original frame
print("data: done solving")
# append best settings to an existing csv file
# best_settings.to_csv("best_settings_for_loop.csv", mode="a", header=False, index=False)
# create_map_with_accessible_tooltips(dataset)


In [ ]:
print(generate_route_statistics(dataset))


In [ ]:
create_map_with_accessible_tooltips(dataset)


In [ ]:
# _ = combined_tracker_and_statistics(solution, filename, dataset, data, manager, routing, search_parameters)
# m = create_map_with_accessible_tooltips(dataset)
# display(m)
# routes_by_days_to_html(dataset)
generate_route_statistics(dataset)


In [ ]:
dataset = assign_new_rts(data, manager, routing, solution, dataset)
_


In [ ]:
dataset = set_days_for_routes(
    dataset,
    min_stops=1,
    max_stops=10,
    distance_threshold=999,  # average_new_dist,
    day_factor=12,
)
# Ensure both datasets have a common identifier, set it as index if not already
# dataset.set_index('CUST', inplace=True)
# sample_route_data.set_index('CUST', inplace=True)

# # Update the dataset with the sample data; this updates rows in 'dataset' with those in 'sample_route_data'
# # based on the common index ('CUST' here)
# dataset.update(sample_route_data)

# # Reset index if necessary, to move 'CUST' back to a column
# dataset.reset_index(inplace=True)

_ = combined_tracker_and_statistics(
    solution, filename, dataset, data, manager, routing, search_parameters)
m = create_map_with_accessible_tooltips(dataset)
# display(m)
routes_by_days_to_html(dataset)


In [ ]:
seconds_per_iter = 19
console = Console()
explore_coefficient = 100_000_000_000
memory_coefficient = 0.7
solve_minutes = 1
starting_explore_factor = 1.3
loops = 100
solution_limit = 2
fps = 20
# create variable rand_bool that is a random boolean


In [ ]:
# The Loop #This is it ### Click ## HERE

# import the rich library progress bar
from rich.progress import track
from rich import print


from rich.console import Console
from rich.table import Table
from rich.progress import track
from rich.panel import Panel
from rich.text import Text
from rich import box
from rich import color
from time import sleep
import datetime
import random
from rich.progress import track, Progress, ProgressColumn, TextColumn, SpinnerColumn, TimeElapsedColumn
old_objective = solution.ObjectiveValue() + 1

gtg = False
# save the current time to a variable
start_time = datetime.datetime.now()


for i in range(loops):

    # solution_limit += 1
    end_time = datetime.datetime.now()
    elapsed_time = end_time - start_time
    now = datetime.datetime.now()
    # print the current time in a human readable format
    now = now.strftime('%I:%M:%S %p')
    print(
        f"Starting iteration {i+1} at {now}... Elapsed time: {elapsed_time}  Explore: {explore_coefficient}  Objective: {solution.ObjectiveValue()}")
    # explore_coefficient = 0#10000000000#int(explore_coefficient / starting_explore_factor)
    # if memory_coefficient <= 0.92:
    #     memory_coefficient += 0.03
    # max_retry = 1
    # if gtg and solution.ObjectiveValue() == old_objective:
    #     seconds_per_iter = int((seconds_per_iter+1)*1.05)
    #     for p in range(max_retry):
    #         p_start_time = datetime.datetime.now()
    #         rand_decimal = random.uniform(0.01, 0.9)
    #         rand_exp = random.randint(2, 9)
    #         # rand_int = random.randint(100, 1000)*11**rand_exp
    #         rand_int = 0#100000000000000
    #         search_parameters = set_search_parameters(search_parameters, solve_minutes=1, explore_coefficient=int(rand_int), memory_coefficient=(rand_decimal), log_search=False, seconds_per_iter=27, solution_limit=solution_limit)

    #         print(f"Iteration: {p} Explore: {rand_int} Memory: {rand_decimal} Objective: {old_objective} New Objective: {solution.ObjectiveValue()}")
    #         old_objective = solution.ObjectiveValue()
    #         search_parameters.solution_limit = solution_limit
    #         solution = routing.SolveFromAssignmentWithParameters(solution, search_parameters)
    #         gtg = False
    #         p_end_time = datetime.datetime.now()
    #         p_duration = p_end_time - p_start_time
    #         print(f"Elapsed time for iteration {i} is {p_duration} with value {solution.ObjectiveValue()}")
    #         if old_objective != solution.ObjectiveValue():
    #             # memory_coefficient = rand_decimal
    #             # explore_coefficient = rand_int
    #             gtg = True
    #             break
    # else:
    # if True:
    search_parameters = set_search_parameters(search_parameters, solve_minutes=solve_minutes, explore_coefficient=int(
        explore_coefficient), memory_coefficient=memory_coefficient, log_search=False, seconds_per_iter=seconds_per_iter, solution_limit=solution_limit)
    old_objective = solution.ObjectiveValue()
    # set a limit for the number of solutions find
    search_parameters.solution_limit = solution_limit
    search_parameters.time_limit.seconds = seconds_per_iter
    search_parameters.use_depth_first_search = pywrapcp.BOOL_TRUE

    search_parameters.multi_armed_bandit_compound_operator_exploration_coefficient = explore_coefficient
    last_obj = solution.ObjectiveValue()
    solution = routing.SolveFromAssignmentWithParameters(
        solution, search_parameters)
    gtg = True
    dataset = assign_new_rts(data, manager, routing, solution, dataset)
    # dset = combined_tracker_and_statistics(solution, filename, dataset, data, manager, routing, search_parameters)
    average_new_dist = (
        dataset.groupby(["NEW RT", "NEW DAY", "NEW STOP"])["NEW DIST"]
        .sum()
        .quantile(0.92)
    )
    #  Random condition that is True 20% of the time
    if random.random() < 0.2:
        explore_coefficient = 0
    else:
        explore_coefficient = 100_000_000_000
    # split the routes into days
    dataset = set_days_for_routes(
        dataset,
        min_stops=5,
        max_stops=9,
        distance_threshold=average_new_dist,
        day_factor=4,
    )
    # if memory_coefficient > .9:
    #     memory_coefficient = 0.01
    # solve_minutes *= 1.05
    # starting_explore_factor += 0.1

    # if old_objective == solution.ObjectiveValue():
    #     # explore_coefficient *= 4
    #     seconds_per_iter = int((seconds_per_iter+1)*1.05)
    #     if memory_coefficient >= 0.1:
    #         memory_coefficient -= 0.1
    # create animation every 4th iter of i
    if last_obj != solution.ObjectiveValue():
        fname = create_final_route_frame(dataset)  # Display the final frame
        filenames.append(fname)
    # display(final_fig[i])
    if i % 5 == 0:
        seconds_per_iter += 2
        solution_limit += 1


dataset = assign_new_rts(data, manager, routing, solution, dataset)

dataset = set_days_for_routes(
    dataset,
    min_stops=5,
    max_stops=8,
    distance_threshold=average_new_dist,
    day_factor=2,
)
# _ = combined_tracker_and_statistics(solution, filename, dataset, data, manager, routing, search_parameters)
# m = create_map_with_accessible_tooltips(dataset)
# display(m)
# create_video_from_filenames(filenames)


def display_video_in_notebook(video_path):
    """
    Display a video file in a Jupyter Notebook.

    Args:
        video_path (str): The path to the video file to display.
    """
    video_html = HTML(f"""
        <video width="640" height="480" controls>
          <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
        </video>
    """)
    display(video_html)


create_video_from_filenames(
    filenames, fps=fps, output_filename=f"rtev{solution.ObjectiveValue()}.mp4")
display_video_in_notebook(f"rtev{solution.ObjectiveValue()}.mp4")


In [ ]:
create_video_from_filenames(
    filenames[60:], fps=2, output_filename=f"rtevakSA{solution.ObjectiveValue()}.mp4")
display_video_in_notebook(f"rtevakSA{solution.ObjectiveValue()}.mp4")


In [ ]:
create_map_with_accessible_tooltips(dataset)


In [ ]:
dataset.to_csv("final_dataset5.csv", index=False)


## old stuff


In [ ]:
create_video_from_filenames(
    filenames, fps=5, output_filename=f"dog{solution.ObjectiveValue()}.mp4")
display_video_in_notebook(f"dog{solution.ObjectiveValue()}.mp4")


In [ ]:
create_video_from_filenames(filenames, fps=3,)
display_video_in_notebook('route_animation.mp4')


In [ ]:
import os
import glob


def clear_image_output_directory(directory):
    """
    Deletes all files in the specified directory.

    Args:
        directory (str): The path to the directory from which to delete files.
    """
    # Construct a path pattern to match all files in the directory
    file_pattern = os.path.join(directory, '*')
    # List all files matching the pattern
    files = glob.glob(file_pattern)
    # Loop over the list of filepaths & remove each file.
    for file in files:
        os.remove(file)
    print(f"All files removed from {directory}")


# Assuming your output directory is named 'route_snapshots'
clear_image_output_directory('route_snapshots')


In [ ]:
create_map_with_accessible_tooltips(dataset)
